# Conception BDD Normalisée et Conforme au RGPD (SIDORA AI)

In [2]:
# import
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

In [ ]:
# Creation structure de base
Base = declarative_base()

#Client
class Age(Base):
    __tablename__ = "ages"
    age_id = Column(Integer, primary_key=True)
    age_plage = Column(String)
    # relationship
    clients = relationship("Client", back_populates=age_id)

class Region(Base):
    __tablename__ = "regions"
    region_id = Column(Integer, primary_key=True)
    region_n = Column(String)
    # relationship
    clients = relationship("Client", back_populates=region_id)

class DonnePersonnel(Base):
    __tablename__ = "donnes_personnels"
    id = Column(Integer, primary_key=True)
    nome = Column(String)
    mot_de_pass = Column(String)
    client_id = Column(Integer, ForeignKey("clients.client_id"))
    # relationship
    clients = relationship("Client", back_populates="client_id")

class Client(Base):
    __tablename__ = "clients"
    client_id = Column(Integer, primary_key=True)
    age_category = Column(Integer, ForeignKey("ages.age_category"))
    regoin_id = Column(Integer, ForeignKey("regions.region_id"))
    # relation
    ages = relationship("Age", back_populates="age_id")
    regions = relationship("Region", back_populates="region_id")


In [ ]:
# Initialisation
engine = create_engine("sqlite:///bd_joue_vent")
SesseonLocal = sessionmaker(bind=engine)
Base.metadata.create_all(engine)

In [ ]:
# Utilisation
session = SesseonLocal

u = DonnePersonnel(name="Hanna", mot_de_pass="pass")

session.add(u)
session.commit()
session.close()